In [1]:
from typing import List, Any
from dotenv import load_dotenv
import json

In [2]:
from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM

In [3]:
class Agent:
    def __init__(self, 
                 role:str = "Personal Assistant", 
                 instructions:str = "Help user with any question",
                 model:str = "gpt-4o-mini",
                 temperature: float = 0,
                 tools: List[Any] = None):
        self.model = model
        self.role = role
        self.instructions = instructions
        self.tools = tools

        load_dotenv() # load enviornment variables
        self.llm = LLM(model=model, temperature=temperature, tools=tools, api_key="voc-739452566168865424292768abb5376f0dd6.20650778")

    def invoke(self, user_message: str) -> str:
        messages = [
            SystemMessage(
                content=(
                    f"You're an AI Agent and your role is {self.role}."
                    f"Your instructions: {self.instructions}"
                )
            )
        ]
        messages.append(UserMessage(
            content=user_message
        ))

        ai_message = self.llm.invoke(messages)
        messages.append(ai_message)

        while ai_message.tool_calls:
            # iterate through all the calls and collect the call responses
            for call in ai_message.tool_calls:
                func_name = call.function.name
                func_args = json.loads(call.function.arguments)
                tool_call_id = call.id

                tool = next((t for t in self.tools if t.name == func_name), None)
                if tool:
                    result = tool(**func_args)
                    messages.append(
                        ToolMessage(
                            content = json.dumps(result),
                            tool_call_id = tool_call_id,
                            name=func_name
                        )
                    )
            ai_message = self.llm.invoke(messages)
            messages.append(ai_message)
        for m in messages:
            print(m)
        return ai_message.content
            

In [4]:
agent = Agent(role="Coding Assistant")
response = agent.invoke("What is Typescript? be concise")
print(response)

content="You're an AI Agent and your role is Coding Assistant.Your instructions: Help user with any question" role='system'
content='What is Typescript? be concise' role='user'
content='TypeScript is a superset of JavaScript that adds static typing to the language. It allows developers to catch errors at compile time, enhances code readability, and improves tooling support. TypeScript code is transpiled to standard JavaScript, making it compatible with any environment that supports JavaScript.' role='assistant' tool_calls=None
TypeScript is a superset of JavaScript that adds static typing to the language. It allows developers to catch errors at compile time, enhances code readability, and improves tooling support. TypeScript code is transpiled to standard JavaScript, making it compatible with any environment that supports JavaScript.


## 1. Basic tool 

In [5]:
@tool
def calculate(expression:str)-> float:
    return eval(expression)
    

In [6]:
math_agent = Agent(
    role="Math Assistant",
    tools=[calculate]
)

In [7]:
response = math_agent.invoke("what is 25*25?")
print(response)

content="You're an AI Agent and your role is Math Assistant.Your instructions: Help user with any question" role='system'
content='what is 25*25?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_3rKRyHBC0y2itANy76YVm9iX', function=Function(arguments='{"expression":"25*25"}', name='calculate'), type='function')]
content='625' role='tool' tool_call_id='call_3rKRyHBC0y2itANy76YVm9iX' name='calculate'
content='The result of \\( 25 \\times 25 \\) is 625.' role='assistant' tool_calls=None
The result of \( 25 \times 25 \) is 625.


In [8]:
response = math_agent.invoke("what is 25 multiply by 25?")

content="You're an AI Agent and your role is Math Assistant.Your instructions: Help user with any question" role='system'
content='what is 25 multiply by 25?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_ONnGgPhaPCVxI4XhDRymPKsb', function=Function(arguments='{"expression":"25 * 25"}', name='calculate'), type='function')]
content='625' role='tool' tool_call_id='call_ONnGgPhaPCVxI4XhDRymPKsb' name='calculate'
content='25 multiplied by 25 is 625.' role='assistant' tool_calls=None


## 2. Tool with mock data

In [16]:
@tool
def get_games(num_games: int=1, top:bool=True):
    data = [
        {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
        {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
        {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
        {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
        {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
        {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
        {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
        {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
        {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
        {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88}
    ]
    sorted_games = sorted(data, key=lambda x: x['Score'], reverse=top)

    return sorted_games[:num_games]

In [17]:
data_analyst_agent = Agent(
    role="Game Data Analyst",
    instructions= "You analyze data about game dataset and answer user questions about the game",
    tools = [get_games]
)

In [18]:
response = data_analyst_agent.invoke("what was the best game in the dataset?")
print(response)

content="You're an AI Agent and your role is Game Data Analyst.Your instructions: You analyze data about game dataset and answer user questions about the game" role='system'
content='what was the best game in the dataset?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_dz3Db3NjSw4798pwIwz9gvKw', function=Function(arguments='{"top":true}', name='get_games'), type='function')]
content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}]' role='tool' tool_call_id='call_dz3Db3NjSw4798pwIwz9gvKw' name='get_games'
content='The best game in the dataset is "The Legend of Zelda: Breath of the Wild" for the Switch, with a score of 98.' role='assistant' tool_calls=None
The best game in the dataset is "The Legend of Zelda: Breath of the Wild" for the Switch, with a score of 98.
